In [ ]:
from spf.dataset.fake_dataset import create_fake_dataset, fake_yaml_v5

# import tempfile
# tmpdir = tempfile.TemporaryDirectory()
# ds_fn = f"{tmpdir.name}/test_circle"
ds_fn = f"./test_circle"

In [ ]:
# generates a complete fake dataset , where the transmitter makes two
# orbits around a stationary listener (with two radios , 4 antennas in cross pattern)
# Simulates a total of 800 points with noise=0.7
# this might take a few minutes
import os

if not os.path.exists(ds_fn + ".zarr"):
    print("simulate data file")
    create_fake_dataset(  # requires ~ 11GB space
        yaml_config_str=fake_yaml_v5,
        filename=ds_fn,
        orbits=2,
        n=800,
        noise=0.7,
    )

In [ ]:
import os
import spf

# this contains the empirical distribution mapping observed phase difference
# to the data distribution of theta (ground truth angle between receiver and transmitter)
empirical_data_fn = os.path.join(
    os.path.dirname(spf.__file__), "../empirical_dists/full.pkl"
)

In [ ]:
# call this to create the "precompute_cache" a file with precomputed
# features from the raw radio signal file that was simulated. These
# are much smaller, easier to load, and required for further analysis/tracking
from spf.dataset.spf_dataset import v5spfdataset

# import tempfile
# tmp_precompute_cache = tempfile.TemporaryDirectory()
# precompute_cache_dir = tmp_precompute_cache.name
precompute_cache_dir = "./"
# ds_fn = "/mnt/md2/rpi5inference/rpi5inference_2025_04_09_03_27_56_nRX2_bounce_spacing0p035.zarr"
ds_fn = "/mnt/md2/rpi5inference/rpi5inference_2025_04_09_15_36_19_nRX2_bounce_spacing0p035.zarr"
precompute_cache_dir = "/mnt/md2/rpi5inference/"
# we load it here just to segment
v5spfdataset(
    ds_fn,
    precompute_cache=precompute_cache_dir,
    nthetas=65,
    skip_fields=set(["signal_matrix"]),  # dont load the full raw signal matrix
    empirical_data_fn=empirical_data_fn,
    paired=True,
    ignore_qc=True,
    gpu=True,  # use GPU in segmentation (pre-processing) # turn off if your GPU cant go VROOM-VROOOM
    temp_file=False,
    temp_file_suffix="",
    segment_if_not_exist=True,
)

print(f"{ds_fn} This file can now also be viewed with the notebook v5dataset_analysis")

In [ ]:
from spf.dataset.spf_dataset import v5spfdataset_manager
from spf.filters.particle_dualradio_filter import plot_single_theta_dual_radio
from spf.filters.run_filters_on_data import run_PF_single_theta_dual_radio


# temp_file=True: Used for accessing **in-progress recordings** where data
# is actively being written. Files are expected to have a temporary extension (default `.tmp`).
# temp_file=False: Used for accessing **completed recordings** where all data
# is finalized and no new data will be added.

temp_file = False

# This shouldnt rebuild the precompute_cache that was created above, once its made
# its re-used

ds = v5spfdataset(
    ds_fn,
    precompute_cache=precompute_cache_dir,
    nthetas=65,
    skip_fields=set(["signal_matrix"]),
    empirical_data_fn=empirical_data_fn,
    paired=True,
    ignore_qc=True,
    gpu=False,
    temp_file=False,
    temp_file_suffix="",
)

# arguments for particle filter
args = {
    "ds": ds,
    "N": 1024 * 4,  # how many particles to simulate
    "theta_err": 0.01,  # the variance in angle estimate
    "theta_dot_err": 0.01,  # the variance in angle velocity estimate
    "progress_bar": True,
}
# this will run a particle filter using the observed probability distribution
# p( theta | phase_difference ) , from real data , this is the part loaded from empirical_data_fn
result = run_PF_single_theta_dual_radio(**args)

_ = plot_single_theta_dual_radio(ds)

In [ ]:
from spf.filters.particle_dualradio_filter import plot_single_theta_dual_radio
from spf.filters.run_filters_on_data import run_PF_single_theta_dual_radio
from spf.dataset.spf_dataset import v5spfdataset

# When temp_file=True: Used for accessing **in-progress recordings** where data
# is actively being written. Files are expected to have a temporary extension (default `.tmp`).
# When temp_file=False: Used for accessing **completed recordings** where all data
# is finalized and no new data will be added.

import tempfile

temp_file = True
tmpdir = tempfile.TemporaryDirectory()

# To profile this run
# py-spy top benchmark_pipeline_runtime.py
# with the contents of runtime.py being: this cell
# looks like detrend_1d_np is taking up most of the time..
# this is a new addition to the pipeline
ds = v5spfdataset(
    ds_fn,
    precompute_cache=tmpdir.name,  # pass in temp dir, dont accidentally pass in other cache
    nthetas=65,
    skip_fields=set(["signal_matrix"]),
    empirical_data_fn=empirical_data_fn,
    paired=True,
    ignore_qc=True,
    gpu=False,  # use GPU in segmentation (pre-processing) # turn off if your GPU cant go VROOM
    temp_file=temp_file,
    temp_file_suffix="",
    n_parallel=0,  # important for live data
    skip_detrend=True,  # this is normally done, buts its way way to slow
)
args = {
    "ds": ds,
    "N": 1024 * 4,
    "theta_err": 0.01,
    "theta_dot_err": 0.01,
    "progress_bar": True,
}
result = run_PF_single_theta_dual_radio(**args)
plot_single_theta_dual_radio(ds)